In [29]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.backend import int_shape
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.models import Model

from tensorflow.keras import layers 
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, SpatialDropout2D, 
                                     UpSampling2D, Input, concatenate, multiply, add, Activation, Cropping2D)

In [30]:
# Tried data from the same distribution, split data from our satelite images into train and validation sets

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict=data_gen_args,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    mask_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'training',
        seed = seed) 
    image_val_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'validation',
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'training',
        seed = seed)
    mask_val_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'validation',
        seed = seed)
    
        
    return image_generator, image_val_generator, mask_generator, mask_val_generator

image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(3,'dataset','images_satelite','labels_satelite',data_gen_args)

def train(image_generator, mask_generator, flag_multi_class = False, num_class = 2):
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)       
        yield (img,mask)
        
def val(image_val_generator, mask_val_generator, flag_multi_class = False, num_class = 2):
    val_generator = zip(image_val_generator, mask_val_generator)
    for (img_val,mask_val) in val_generator:
        img_val,mask_val = adjustData(img_val,mask_val,flag_multi_class,num_class)       
        yield (img_val,mask_val)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

Found 12 images belonging to 1 classes.
Found 2 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 2 images belonging to 1 classes.


Train form data in dataset using data augmentation

In [31]:
# Vanilla unet

In [32]:

def get_crop_shape( target, refer):

    # width, the 3rd dimension
    cw = (target.get_shape()[2] - refer.get_shape()[2])

    assert (cw >= 0)

    if cw % 2 != 0:

        cw1, cw2 = int(cw/2), int(cw/2) + 1

    else:

        cw1, cw2 = int(cw/2), int(cw/2)

    # height, the 2nd dimension

    ch = (target.get_shape()[1] - refer.get_shape()[1])

    assert (ch >= 0)

    if ch % 2 != 0:

        ch1, ch2 = int(ch/2), int(ch/2) + 1

    else:

        ch1, ch2 = int(ch/2), int(ch/2)



    return (ch1, ch2), (cw1, cw2)

def VanillaUnet( num_class = 1, img_shape = (256,256,3)):

    concat_axis = 3

    # input
    
    inputs = layers.Input(shape = img_shape)


    # Unet convolution block 1
    
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(inputs)

    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)

    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    

    # Unet convolution block 2

    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)

    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)

    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)



    # Unet convolution block 3

    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)



    # Unet convolution block 4

    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)

    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    print(conv4.shape)

    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)



    # Unet convolution block 5

    conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)

    conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)



    # Unet up-sampling block 1; Concatenation with crop_conv4

    up_conv5 = layers.UpSampling2D(size=(2, 2))(conv5)
    print(up_conv5.shape)

    ch, cw = get_crop_shape(conv4, up_conv5)

    crop_conv4 = layers.Cropping2D(cropping=(ch,cw))(conv4)

    up6 = layers.concatenate([up_conv5, crop_conv4], axis=concat_axis)

    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up6)

    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)



    # Unet up-sampling block 2; Concatenation with crop_conv3

    up_conv6 = layers.UpSampling2D(size=(2, 2))(conv6)

    ch, cw = get_crop_shape(conv3, up_conv6)

    crop_conv3 = layers.Cropping2D(cropping=(ch,cw))(conv3)

    up7 = layers.concatenate([up_conv6, crop_conv3], axis=concat_axis) 

    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up7)

    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)



    # Unet up-sampling block 3; Concatenation with crop_conv2

    up_conv7 = layers.UpSampling2D(size=(2, 2))(conv7)

    ch, cw = get_crop_shape(conv2, up_conv7)

    crop_conv2 = layers.Cropping2D(cropping=(ch,cw))(conv2)

    up8 = layers.concatenate([up_conv7, crop_conv2], axis=concat_axis)

    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up8)

    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)



    # Unet up-sampling block 4; Concatenation with crop_conv1

    up_conv8 = layers.UpSampling2D(size=(2, 2))(conv8)

    ch, cw = get_crop_shape(conv1, up_conv8)

    crop_conv1 = layers.Cropping2D(cropping=(ch,cw))(conv1)

    up9 = layers.concatenate([up_conv8, crop_conv1], axis=concat_axis)

    conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up9)

    conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)



    ch, cw = get_crop_shape(inputs, conv9)

    conv9 = layers.ZeroPadding2D(padding=((ch[0], ch[1]), (cw[0], cw[1])))(conv9)

    conv10 = layers.Conv2D(num_class, (1, 1))(conv9)
    print(conv10.shape)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [36]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')

model = VanillaUnet()
model_checkpoint = ModelCheckpoint('vanilla_unet_split.hdf5', monitor='val_accuracy',verbose=1, save_best_only=False)
model.compile(optimizer=Adam(), loss='binary_crossentropy',metrics=['accuracy'])
model.fit_generator(train_generator,steps_per_epoch=800,epochs=5,validation_data=val_generator,validation_steps=120, callbacks=[model_checkpoint])

(None, 32, 32, 256)
(None, 32, 32, 512)
(None, 256, 256, 1)
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 800 steps, validate for 120 steps
Epoch 1/5
799/800 [============================>.] - ETA: 0s - loss: 0.4127 - accuracy: 0.7921
Epoch 00001: saving model to vanilla_unet_split.hdf5
800/800 [==============================] - 80s 101ms/step - loss: 0.4126 - accuracy: 0.7921 - val_loss: 0.4574 - val_accuracy: 0.7260
Epoch 2/5
799/800 [============================>.] - ETA: 0s - loss: 0.4326 - accuracy: 0.7801
Epoch 00002: saving model to vanilla_unet_split.hdf5
800/800 [==============================] - 79s 99ms/step - loss: 0.4326 - accuracy: 0.7801 - val_loss: 0.4291 - val_accuracy: 0.8242
Epoch 3/5
799/800 [============================>.] - ETA: 0s - loss: 0.5351 - accuracy: 0.7342
Epoch 00003: saving model to vanilla_unet_split.hdf5
800/800 [==============================] - 80s 100ms/step - loss: 0.5352 - accuracy: 0.7341 - val_loss: 0.4306 - val_accuracy: 0.8218
E

Test model

In [ ]:
model=satellite_unet(pretrained='sat_unet.hdf5')
#testGene = testGenerator("dataset/unlabelled")
#results = model.predict_generator(testGene,30,verbose=1)
X_test,names=test("dataset/unlabelled")
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults("dataset/preds",results,names)

In [40]:
def test(test_path,target_size=(256,256), color_mode = "rgb"):
    X_test=[]
    names=[]
    for filename in os.listdir(test_path):
        name, ext = os.path.splitext(filename)
        if ext!=".png" and ext!=".jpg":
            continue
        names.append(filename)
        img=load_img(os.path.join(test_path,filename),target_size=(256,256), color_mode = color_mode)
        img=img_to_array(img)/255
        X_test.append(img.copy())
    X_test_label = np.array(X_test)
    return X_test_label

test_img = test('dataset/images_satelite')

print(test_img.shape)

test_label = test('dataset/labels_satelite', color_mode = "grayscale")
print(test_label.shape)


(14, 256, 256, 3)
(14, 256, 256, 1)


In [46]:
model=satellite_unet(pretrained='vanilla.hdf5')

score, acc = model.evaluate(test_img, test_label)
print('Test score:', score)
print('Test accuracy:', acc)

18/18 [==============================] - 1s 52ms/sample - loss: 1.5818 - accuracy: 0.6035
Test score: 1.581809163093567
Test accuracy: 0.6035216


In [5]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(3,'dataset','images','labels',data_gen_args,save_to_dir = None)
model = satellite_unet()
model_checkpoint = ModelCheckpoint('sat_unet_IoU.hdf5', monitor='acc',verbose=1, save_best_only=False)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

Found 27 images belonging to 1 classes.
Found 27 images belonging to 1 classes.
  ...
    to  
  ['...']
Train for 2000 steps
Epoch 1/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.2735 - mean_io_u_1: 0.3362
Epoch 00001: saving model to sat_unet_IoU.hdf5
2000/2000 [==============================] - 274s 137ms/step - loss: 0.2735 - mean_io_u_1: 0.3362
Epoch 2/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.1675 - mean_io_u_1: 0.3360
Epoch 00002: saving model to sat_unet_IoU.hdf5
2000/2000 [==============================] - 270s 135ms/step - loss: 0.1675 - mean_io_u_1: 0.3361
Epoch 3/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.1270 - mean_io_u_1: 0.3361
Epoch 00003: saving model to sat_unet_IoU.hdf5
2000/2000 [==============================] - 270s 135ms/step - loss: 0.1269 - mean_io_u_1: 0.3362
Epoch 4/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.1024 - mean_io_u_1: 0.3364
Epoch 00004: saving model to sat_unet_I

In [41]:
model=satellite_unet(pretrained='sat_unet_.hdf5')

score, acc = model.evaluate(test_img, test_label)
print('Test score:', score)
print('Test accuracy:', acc)

14/14 [==============================] - 2s 166ms/sample - loss: 0.5189 - accuracy: 0.7684
Test score: 0.518913209438324
Test accuracy: 0.7684392


In [ ]:
def conv2d_block(
    inputs,
    use_batch_norm=True,
    dropout=0.3,
    dropout_type="spatial",
    filters=16,
    kernel_size=(3, 3),
    activation="relu",
    kernel_initializer="he_normal",
    padding="same",
):

    if dropout_type == "spatial":
        DO = SpatialDropout2D
    elif dropout_type == "standard":
        DO = Dropout
    else:
        raise ValueError(f"dropout_type must be one of ['spatial', 'standard'], got {dropout_type}")

    c = Conv2D(filters,kernel_size,activation=activation,kernel_initializer=kernel_initializer,
        padding=padding,use_bias=not use_batch_norm,)(inputs)
    
    if use_batch_norm:
        c = BatchNormalization()(c)
    if dropout > 0.0:
        c = DO(dropout)(c)
    c = Conv2D(filters, kernel_size, activation=activation, kernel_initializer=kernel_initializer,
        padding=padding, use_bias=not use_batch_norm,)(c)
    if use_batch_norm:
        c = BatchNormalization()(c)
    return c

In [ ]:
def vanilla_unet(
    input_shape=(256,256,3),
    num_classes=1,
    dropout=0.5, 
    filters=64,
    num_layers=4,
    output_activation='sigmoid'): # 'sigmoid' or 'softmax'

    # Build U-Net model
    inputs = Input(input_shape)
    x = inputs   
    print('1', x.shape)

    down_layers = []
    for l in range(num_layers):
        x = conv2d_block(inputs=x, filters=filters, use_batch_norm=False, dropout=0.0, padding='valid')
        print('l', x.shape)
        down_layers.append(x)
        x = MaxPooling2D((2, 2), strides=2) (x)
        print('l', x.shape)
        filters = filters*2 # double the number of filters with each layer

    x = Dropout(dropout)(x)
    x = conv2d_block(inputs=x, filters=filters, use_batch_norm=False, dropout=0.0, padding='valid')
    print('11', x.shape)

    for conv in reversed(down_layers):
        
        filters //= 2 # decreasing number of filters with each layer 
        x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='valid') (x)
        print('conv1', x.shape)
        
        ch, cw = get_crop_shape(int_shape(conv), int_shape(x))
        conv = Cropping2D(cropping=(ch, cw))(conv)
        print('conv2', conv.shape)

        x = concatenate([x, conv])
        print('concatenate',x.shape)
        x = conv2d_block(inputs=x, filters=filters, use_batch_norm=False, dropout=0.0, padding='valid')
        print('final', x.shape)
    
    outputs = Conv2D(num_classes, (1, 1), activation=output_activation) (x)    
    print('output', outputs.shape)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


def get_crop_shape(target, refer):
    # width, the 3rd dimension
    cw = target[2] - refer[2]
    assert (cw >= 0)
    if cw % 2 != 0:
        cw1, cw2 = int(cw/2), int(cw/2) + 1
    else:
        cw1, cw2 = int(cw/2), int(cw/2)
    # height, the 2nd dimension
    ch = target[1] - refer[1]
    assert (ch >= 0)
    if ch % 2 != 0:
        ch1, ch2 = int(ch/2), int(ch/2) + 1
    else:
        ch1, ch2 = int(ch/2), int(ch/2)

    return (ch1, ch2), (cw1, cw2)